In [0]:
import numpy as np
import pandas as pd
import os

In [0]:
train_file = pd.read_csv("train.csv")
dances = train_file.target.unique()
train_file.columns

Index(['Image', 'target'], dtype='object')

In [0]:
current_directory = os.getcwd()
for i in dances:
    final_directory = os.path.join(current_directory, 'final_data', i)
    if not os.path.exists(final_directory):
       os.makedirs(final_directory)

In [0]:
images = train_file['Image']
target = train_file['target']

In [0]:
import shutil

In [0]:
for i in range(0, len(images)):
    shutil.move(current_directory + '/dataset/train/'+str(images[i]), 
                                                          current_directory+'/final_data/'+'/'+target[i]+'/'+str(images[i]))
    print(images[i], target[i])

FileNotFoundError: [Errno 2] No such file or directory: '/home/satyam/Desktop/hackerearth_challenge/dance/deep_learning/dataset/train/96.jpg'

In [5]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
IMAGE_SIZE = [224, 224]

In [7]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [0]:
# useful for getting number of classes
folders = glob('final_data/*')

In [0]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [0]:
model = Model(inputs=vgg.input, outputs=prediction)

In [18]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [20]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('final_data/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

# test_set = test_datagen.flow_from_directory('dataset/test/',
#                                             target_size = (224, 224),
#                                             batch_size = 32)

Found 364 images belonging to 8 classes.


In [21]:
r = model.fit_generator(
  training_set,
  #validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set)
 # validation_steps=len(test_set)
)

Epoch 1/10
12/12 [==============================] - 5s 452ms/step - loss: 0.4967 - accuracy: 0.8297
Epoch 2/10
12/12 [==============================] - 5s 448ms/step - loss: 0.1831 - accuracy: 0.9451
Epoch 3/10
12/12 [==============================] - 5s 440ms/step - loss: 0.1061 - accuracy: 0.9753
Epoch 4/10
12/12 [==============================] - 5s 437ms/step - loss: 0.0840 - accuracy: 0.9725
Epoch 5/10
12/12 [==============================] - 5s 439ms/step - loss: 0.0662 - accuracy: 0.9918
Epoch 6/10
12/12 [==============================] - 5s 439ms/step - loss: 0.0560 - accuracy: 0.9808
Epoch 7/10
12/12 [==============================] - 5s 442ms/step - loss: 0.0413 - accuracy: 0.9918
Epoch 8/10
12/12 [==============================] - 5s 434ms/step - loss: 0.0479 - accuracy: 0.9863
Epoch 9/10
12/12 [==============================] - 5s 438ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 10/10
12/12 [==============================] - 5s 440ms/step - loss: 0.0127 - accuracy: 1.0000

In [0]:
import tensorflow as tf
from keras.models import load_model

model.save('my_model.h5')